# Znajdowanie maksymalnego profitu

### Implementacja #1
##### Z użyciem kolejki priorytetowej

In [1]:
class MaxPriorityQueue:
    def __init__(self, maxsize=128):
        if not isinstance(maxsize, int):
            raise TypeError(f"expected 'int', got {str(type(maxsize))[7:-1]}")
        if maxsize <= 0:
            raise ValueError(f"cannot create a {self.__class__.__name__} of a max size {maxsize}")
        self.heap = [None] * maxsize  # Allocate a constant memory space
        self._free_idx = 0

    def __bool__(self):
        return bool(self._free_idx)
    
    def __len__(self):
        return self._free_idx
    
    @property
    def heap_size(self):
        return len(self.heap)
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def insert(self, priority, obj: object):
        if len(self) == self.heap_size:
            raise OverflowError(f'insert in a completely filled {self.__class__.__name__}')
        # Add a value as the last node of our Complete Binary Tree
        self.heap[self._free_idx] = (priority, obj)
        # Fix heap in order to satisfy a min-heap property
        self._free_idx += 1
        self._heapify_up(len(self) - 1)
        
    def get_first(self) -> ('priority', object):
        return None if not self.heap else self.heap[0]  # Return a priority-element pair
        
    def remove_first(self) -> ('priority', object):
        if len(self) == 0:
            raise IndexError(f'remove_min from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self.heap[0]
        # Place the last leaf in the root position
        last_idx = self._free_idx - 1
        last = self.heap[last_idx]
        self.heap[last_idx] = None
        self._free_idx -= 1
        if len(self) > 0:
            self.heap[0] = last
            # Fix a heap in order to stisfy a min-heap property
            self._heapify_down(0, len(self))
        return removed  # Return a priority-element pair
    
    def _swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            # Compare the priority of elements and move up the element
            # of a lower priority (if it is below an element of a higher priority)
            if self.heap[curr_idx][0] > self.heap[parent_idx][0]:
                self._swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):
        # Loop till the current node has a child of a smaller priority than 
        # itself. We assume that when we enter a node which both children 
        # have larger priority than this node, a subtree which a current node 
        # is a root of must fulfill a min-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            smallest_idx = curr_idx

            if l < end_idx:
                if self.heap[l][0] > self.heap[curr_idx][0]: 
                    smallest_idx = l
                if r < end_idx and self.heap[r][0] > self.heap[smallest_idx][0]:
                    smallest_idx = r

            if smallest_idx != curr_idx:
                self._swap(curr_idx, smallest_idx)
                curr_idx = smallest_idx
            else:
                break

def knapsack(P: 'array of profits', W: 'array of weights', MaxW: 'max weight of knapsack'):
    n = len(P)
    # Create a priority queue where a priority is a profit/weight ratio
    pq = MaxPriorityQueue(n)
    for i in range(n):
        pq.insert(P[i] / W[i], i)
    # Take elements of the maximum profit/weight ratio till there is still
    # some space remaining in the knapsack
    w = MaxW
    p = 0
    while w > 0 and pq:
        ratio, i = pq.remove_first()
        # Take the whole item if we can do so
        if W[i] <= w:
            p += P[i]
            w -= W[i]
        # ELse, take only a part of an item which can fit in a knapsack
        else:
            p += ratio * w
            break
            
    # Return a final profit
    return p

###### Kilka testów

In [2]:
P = [7, 2, 5, 10, 8, 1, 2]
W = [3, 1, 6, 3, 6, 10, 2]
MaxW = 10

print(knapsack(P, W, MaxW))

23.0


### Implementacja #2
##### Z użyciem sortowania

In [3]:
def knapsack(P: 'array of profits', W: 'array of weights', MaxW: 'max weight of knapsack'):
    n = len(P)
    # Create an array which we will sort by a profit/weight ratio
    items = [()] * n
    for i in range(n):
        items[i] = (P[i] / W[i], i)
    items.sort(key=lambda x: x[0], reverse=True)
    # Take elements of the maximum profit/weight ratio till there is still
    # some space remaining in the knapsack
    w = MaxW
    p = 0
    i = 0
    while w > 0 and i < n:
        j = items[i][1]
        # Take the whole item if we can do so
        if W[j] <= w:
            p += P[j]
            w -= W[j]
        # ELse, take only a part of an item which can fit in a knapsack
        else:
            ratio = items[i][0]
            p += ratio * w
            break
        # Go to the next most profitable item
        i += 1
            
    # Return a final profit
    return p

###### Kilka testów

In [4]:
P = [7, 2, 5, 10, 8, 1, 2]
W = [3, 1, 6, 3, 6, 10, 2]
MaxW = 10

print(knapsack(P, W, MaxW))

23.0


# Otrzymywanie zwartości

Konieczna jest modyfikacja algorytmu w taki sposób, aby zapisywane były indeksy tych przedmiotów, które zostały wzięte wraz z ich procentową ilością ($ 1 $ - wzięliśmy cały element, $ < 1 $ - wzięliśmy fragment przedmiotu (zawsze będzie maksymalnie jeden taki przedmiot)).

### Implementacja #1
##### Z użyciem kolejki priorytetowej

W poniższej implementacji nie wklejam ponownie kodu, tworzącego kolejkę priorytetową. Aby poniższe funkcja działała prawidłowo, w celu jej użycia poza tym notesem, konieczne jest skopiowanie również wyżej zaimplementowanej klasy $ MaxPriorityQueue $.

In [5]:
def knapsack(P: 'array of profits', W: 'array of weights', MaxW: 'max weight of knapsack'):
    n = len(P)
    # Create a priority queue where a priority is a profit/weight ratio
    pq = MaxPriorityQueue(n)
    for i in range(n):
        pq.insert(P[i] / W[i], i)
    # Take elements of the maximum profit/weight ratio till there is still
    # some space remaining in the knapsack
    w = MaxW
    p = 0
    items = []
    while w > 0 and pq:
        ratio, i = pq.remove_first()
        # Take the whole item if we can do so
        if W[i] <= w:
            p += P[i]
            w -= W[i]
            items.append((i, 1))
        # ELse, take only a part of an item which can fit in a knapsack
        else:
            p += ratio * w
            items.append((i, w / W[i]))
            break
            
    # Return a final profit
    return p, items

###### Kilka testów

In [6]:
P = [7, 2, 5, 10, 8, 1, 2]
W = [3, 1, 6, 3, 6, 10, 2]
MaxW = 10

profit, items = knapsack(P, W, MaxW)
print('Profit gained:', profit)
print('Items taken:')
for i, perc in items:
    print(f'{[i]} Profit: {P[i]:<3} Weight: {W[i]:<3} - {"took the entire item" if perc == 1 else "took {:.2f}% of the item".format(perc * 100)}')

Profit gained: 23.0
Items taken:
[3] Profit: 10  Weight: 3   - took the entire item
[0] Profit: 7   Weight: 3   - took the entire item
[1] Profit: 2   Weight: 1   - took the entire item
[4] Profit: 8   Weight: 6   - took 50.00% of the item


### Implementacja #2
##### Z użyciem sortowania

In [7]:
def knapsack(P: 'array of profits', W: 'array of weights', MaxW: 'max weight of knapsack'):
    n = len(P)
    # Create an array which we will sort by a profit/weight ratio
    items = [()] * n
    for i in range(n):
        items[i] = (P[i] / W[i], i)
    items.sort(key=lambda x: x[0], reverse=True)
    # Take elements of the maximum profit/weight ratio till there is still
    # some space remaining in the knapsack
    w = MaxW
    p = 0
    i = 0
    contents = []
    while w > 0 and i < n:
        j = items[i][1]
        # Take the whole item if we can do so
        if W[j] <= w:
            p += P[j]
            w -= W[j]
            contents.append((j, 1))
        # ELse, take only a part of an item which can fit in a knapsack
        else:
            ratio = items[i][0]
            p += ratio * w
            contents.append((j, w / W[j]))
            break
        # Go to the next most profitable item
        i += 1
            
    # Return a final profit
    return p, contents

###### Kilka testów

In [8]:
P = [7, 2, 5, 10, 8, 1, 2]
W = [3, 1, 6, 3, 6, 10, 2]
MaxW = 10

profit, items = knapsack(P, W, MaxW)
print('Profit gained:', profit)
print('Items taken:')
for i, perc in items:
    print(f'{[i]} Profit: {P[i]:<3} Weight: {W[i]:<3} - {"took the entire item" if perc == 1 else "took {:.2f}% of the item".format(perc * 100)}')

Profit gained: 23.0
Items taken:
[3] Profit: 10  Weight: 3   - took the entire item
[0] Profit: 7   Weight: 3   - took the entire item
[1] Profit: 2   Weight: 1   - took the entire item
[4] Profit: 8   Weight: 6   - took 50.00% of the item
